In [1]:
import pandas as pd  # For handling CSV and Excel file operations
import os  # For file path and existence checks
import tkinter as tk  # For creating the GUI
from tkinter import filedialog, messagebox, ttk  # For file dialogs and additional GUI components
from tkinterdnd2 import TkinterDnD, DND_FILES  # For drag-and-drop functionality (install via `pip install tkinterdnd2`)

def csv_to_excel_gui():
    # Create the main Tkinter window with drag-and-drop support
    root = TkinterDnD.Tk()
    root.title("CSV to Excel Converter")  # Set the window title
    root.geometry("700x500")  # Set the initial size of the window
    root.resizable(True, True)  # Allow resizing of the window
    root.config(bg="#f4f4f9")  # Set background color

    # Variable to track fullscreen state
    is_fullscreen = False

    # Function to toggle fullscreen mode on or off
    def toggle_fullscreen(event=None):
        nonlocal is_fullscreen
        is_fullscreen = not is_fullscreen  # Toggle the state
        root.attributes("-fullscreen", is_fullscreen)  # Enable/disable fullscreen

    # Function to exit fullscreen mode
    def exit_fullscreen(event=None):
        nonlocal is_fullscreen
        is_fullscreen = False  # Reset fullscreen state
        root.attributes("-fullscreen", False)  # Disable fullscreen

    # Function to open a file dialog for selecting a CSV file
    def select_csv_file():
        file_path = filedialog.askopenfilename(
            title="Select a CSV file",
            filetypes=(("CSV files", "*.csv"), ("All files", "*.*")),  # Allow only CSV files
        )
        if file_path:
            load_file(file_path)  # Pass the selected file to the load function

    # Function to handle drag-and-drop files
    def on_file_drop(event):
        file_path = event.data.strip()  # Get the file path from the event
        if os.path.isfile(file_path) and file_path.endswith(".csv"):  # Check if it's a valid CSV file
            load_file(file_path)
        else:
            messagebox.showerror("Error", "Invalid file type. Please drop a valid CSV file.")

    # Function to load the selected file path and display its content
    def load_file(file_path):
        entry_csv.delete(0, tk.END)  # Clear any existing text in the entry box
        entry_csv.insert(0, file_path)  # Insert the selected file path
        preview_csv_file(file_path)  # Preview the contents of the CSV file

    # Function to preview the first 10 rows of the CSV file
    def preview_csv_file(file_path):
        try:
            data = pd.read_csv(file_path, nrows=10)  # Read the first 10 rows
            preview_text.delete(1.0, tk.END)  # Clear any existing preview text
            preview_text.insert(tk.END, data.to_string(index=False))  # Display the data
        except Exception as e:
            messagebox.showerror("Error", f"Could not preview the file. Error: {e}")
            preview_text.delete(1.0, tk.END)
            preview_text.insert(tk.END, "Unable to preview the file.")

    # Function to save the CSV file as an Excel file
    def save_excel_file():
        csv_file_path = entry_csv.get().strip()  # Get the file path from the entry box

        # Check if the file exists and is a valid CSV
        if not os.path.exists(csv_file_path) or not csv_file_path.endswith('.csv'):
            messagebox.showerror("Error", "The file does not exist or is not a CSV file. Please provide a valid CSV file.")
            return

        # Open a file dialog to select where to save the Excel file
        excel_file_path = filedialog.asksaveasfilename(
            defaultextension=".xlsx",
            filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*")),
            title="Save Excel as",
        )
        if not excel_file_path:
            return

        # Start progress bar animation
        progress_bar.start()
        status_label.config(text="⏳ Converting... Please wait.", fg="orange")
        root.update()

        try:
            # Read the CSV and save it as an Excel file
            data = pd.read_csv(csv_file_path)
            data.to_excel(excel_file_path, index=False)
            progress_bar.stop()  # Stop progress bar
            status_label.config(text=f"✅ Success! Saved at: {excel_file_path}", fg="green")
        except Exception as e:
            progress_bar.stop()
            messagebox.showerror("Error", f"❌ An error occurred during the conversion: {e}")
            status_label.config(text="❌ Conversion failed.", fg="red")

    # UI Components
    label_csv = tk.Label(root, text="Drag and Drop a CSV file or Browse:", bg="#f4f4f9", font=("Helvetica", 12))
    label_csv.pack(pady=10)

    entry_csv = tk.Entry(root, width=50, font=("Helvetica", 12))  # Entry box for the file path
    entry_csv.pack(pady=5)

    button_browse = tk.Button(
        root, text="Browse", width=15, height=1, bg="#4CAF50", fg="white", command=select_csv_file
    )  # Button to open file dialog
    button_browse.pack(pady=5)

    button_convert = tk.Button(
        root, text="Convert to Excel", width=20, height=2, bg="#2196F3", fg="white", command=save_excel_file
    )  # Button to convert the file
    button_convert.pack(pady=20)

    # Add file preview area
    preview_label = tk.Label(root, text="File Preview (First 10 rows):", bg="#f4f4f9", font=("Helvetica", 12))
    preview_label.pack(pady=10)

    preview_text = tk.Text(root, height=10, width=80, wrap="none", bg="#f0f0f0", font=("Courier", 10))  # Text widget to show file preview
    preview_text.pack(pady=5)

    # Scrollbars for the preview area
    scroll_x = ttk.Scrollbar(root, orient="horizontal", command=preview_text.xview)
    scroll_x.pack(fill="x", padx=5)
    scroll_y = ttk.Scrollbar(root, orient="vertical", command=preview_text.yview)
    scroll_y.pack(side="right", fill="y")

    preview_text.configure(xscrollcommand=scroll_x.set, yscrollcommand=scroll_y.set)

    # Progress bar widget
    progress_bar = ttk.Progressbar(root, orient="horizontal", length=400, mode="indeterminate")
    progress_bar.pack(pady=10)

    # Status Label
    status_label = tk.Label(root, text="Choose a file to get started.", bg="#f4f4f9", font=("Helvetica", 10, "italic"))
    status_label.pack(pady=10)

    # Drag-and-drop bindings
    root.drop_target_register(DND_FILES)  # Register the window to accept drag-and-drop
    root.dnd_bind('<<Drop>>', on_file_drop)  # Bind the drag-and-drop event

    # Keybindings for fullscreen toggle and exit
    root.bind("<F11>", toggle_fullscreen)
    root.bind("<Escape>", exit_fullscreen)

    # Start the Tkinter event loop
    root.mainloop()


# Run the converter
csv_to_excel_gui()
